In [ ]:
import os
import myServices 
import randForReg as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
DS = pd.read_csv('/Users/abdielfer/DESS/Internship2022/RNCanWork/datasetBasin1.csv')

In [ ]:
DS.isna().sum()

In [ ]:
DS.info()

In [ ]:
DS['percentage'].unique()

In [ ]:
DS['percentage'].value_counts()

In [ ]:
# DS['weights'] = DS['percentage'].map({0:3, 1: 60, 5: 37})
sample = DS.sample(n=10000) # or use : frac=0.05
print(sample.head)
sample['percentage'].unique()

In [ ]:
sample['percentage'] = sample['percentage']*10

In [ ]:
print(sample['percentage'].unique())

In [ ]:
sample['percentage'].value_counts()

In [ ]:
sample.drop(columns=['x_coord', 'y_coord'],inplace = True)
print(sample.head)

In [ ]:
sample.to_csv('./sample.csv')

In [4]:
#Data import and exploration
datasetPath = './sample.csv'
rfReg  = rfr.implementRandomForestRegressor(datasetPath,'percentage', 0.2)
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()

In [ ]:
with timeit():
    bestEstimator = rfReg.fitRFRegressorWeighted(False)

In [6]:
# import executeRFReg as exe
with timeit():
    rfr.main()

 (8000, 7)   : (2000, 7)
Label balance on Training set:  
 0     7827
50     142
10      31
Name: percentage, dtype: int64
Label balance on Validation set:  
 0     1956
50      33
10      11
Name: percentage, dtype: int64
Fitting 3 folds for each of 8160 candidates, totalling 24480 fits
runtime: 0:34:24.176337
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_8298/2701297466.py", line 3, in <cell line: 2>
    rfr.main()
  File "/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/randForReg.py", line 178, in main
    return name
  File "/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/randForReg.py", line 64, in fitRFRegressorWeighted
    weights = createWeightVector(y_train, dominantClassPenalty)
TypeError: saveModel() missing 1 required positional argument: 'name'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/venv/lib/python3.9/site-packages/IPython/core/inter